<a href="https://colab.research.google.com/github/Jacksonngugi/book-recommendation-system/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-11-16 13:31:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  66.9MB/s    in 0.4s    

2023-11-16 13:31:59 (66.9 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df = df_ratings.merge(df_books,on='isbn',how='left')
df

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather
...,...,...,...,...,...
1149775,276704,1563526298,9.0,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard
1149776,276706,0679447156,0.0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil
1149777,276709,0515107662,10.0,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter
1149778,276721,0590442449,10.0,Fourth Grade Rats,Jerry Spinelli


In [ ]:
users = df['user'].value_counts()
isbn = df['isbn'].value_counts()

users = users[users >= 200].index
isbn = isbn[isbn >= 100].index

In [ ]:
#create a new dataframe

df = df.loc[(df['user'].isin(users.values)) & (df['isbn'].isin(isbn.values))]
df = df.drop_duplicates(['title','user'])

df.isna().sum()

#drop isbn without titles and author

df = df.dropna(how='any')

df

,user,isbn,rating,title,author
1456,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1469,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
1471,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
1474,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
1484,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff
...,...,...,...,...,...
1147304,275970,0804111359,0.0,Secret History,DONNA TARTT
1147436,275970,140003065X,0.0,A Fine Balance,Rohinton Mistry
1147439,275970,1400031346,0.0,The No. 1 Ladies' Detective Agency,Alexander McCall Smith
1147440,275970,1400031354,0.0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith


In [ ]:
#create a pivot table
df_pivot = df.pivot(index='title',columns='user',values='rating').fillna(0)
df_pivot

#create a sparse metrix
df_matrix = csr_matrix(df_pivot.values)

In [ ]:
# add your code here - consider creating a new cell for each section of code
model = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)
model.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  #list to put model output
  recommended_books = [book,[]]

  distance, book_info = model.kneighbors([df_pivot.loc[book]], 6, return_distance=True)
  rec_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
  rec_distance = list(np.flip(distance[0])[:-1])

  for r in zip(rec_book_info,rec_distance):
    recommended_books[1].append(list(r))


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
